In [1]:
from music21 import *
import os

In [2]:
notes=[]
notes_to_parse = None
for file in os.listdir("./data"):    
    #parsing a midi file
    midi = converter.parse(f"./data/{file}")
  
    #grouping based on different instruments
    s2 = instrument.partitionByInstrument(midi)

    #Looping over all the instruments
    for part in s2.parts:
    
        #select elements of only piano
        if 'Piano' in str(part): 
        
            notes_to_parse = part.recurse() 
      
            #finding whether a particular element is note or a chord
            for element in notes_to_parse:
                
                #note
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                
                #chord
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder))

In [3]:
x = []
y = []
num_of_steps = 32
for i in range(0, len(notes)-32-1):
    x.append(notes[i:i+num_of_steps])
    y.append(notes[i+num_of_steps])

In [4]:
import numpy as np
x =np.array(x)
y =np.array(y)

In [5]:
unique_x = list(set(x.ravel()))
x_note_to_int = dict((note_, number) for number, note_ in enumerate(unique_x))

In [9]:
import pickle
with open("unique_notes.pkl","wb") as p:
    pickle.dump(unique_x,p)
    p.close()

In [6]:
unique_y = list(set(y))
y_note_to_int = dict((note_, number) for number, note_ in enumerate(unique_y)) 

In [7]:
#preparing input sequences
x_seq=[]
for i in x:
    temp=[]
    for j in i:
        #assigning unique integer to every note
        temp.append(x_note_to_int[j])
    x_seq.append(temp)
    
x_seq = np.array(x_seq)
y_seq=np.array([y_note_to_int[i] for i in y])

In [8]:
y_seq

array([155, 155, 155, ..., 113, 118, 234])

In [9]:
from sklearn.model_selection import train_test_split
x_tr, x_val, y_tr, y_val = train_test_split(x_seq,y_seq,test_size=0.2,random_state=0)

In [10]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation, LSTM
model = Sequential()
model.add(LSTM(128,return_sequences=True))
model.add(LSTM(128))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dense(len(unique_y)))
model.add(Activation('softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
model.build((None,x_tr.shape[1],1))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32, 128)           66560     
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 256)               33024     
_________________________________________________________________
activation (Activation)      (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 358)               92006     
_________________________________________________________________
activation_1 (Activation)    (None, 358)               0         
Total params: 323,174
Trainable params: 323,174
Non-trainable params: 0
__________________________________________________

In [11]:
history = model.fit(np.array(x_tr.reshape(-1,32,1)),np.array(y_tr),batch_size=128,epochs=50,verbose=1)

Epoch 1/50
365/365 [==============================] - 16s 28ms/step - loss: 4.6945
Epoch 2/50
365/365 [==============================] - 11s 29ms/step - loss: 4.5140
Epoch 3/50
365/365 [==============================] - 11s 29ms/step - loss: 4.3205
Epoch 4/50
365/365 [==============================] - 11s 29ms/step - loss: 4.1343
Epoch 5/50
365/365 [==============================] - 11s 29ms/step - loss: 3.9631
Epoch 6/50
365/365 [==============================] - 11s 29ms/step - loss: 3.8140
Epoch 7/50
365/365 [==============================] - 11s 29ms/step - loss: 3.6648
Epoch 8/50
365/365 [==============================] - 11s 29ms/step - loss: 3.5238
Epoch 9/50
365/365 [==============================] - 11s 29ms/step - loss: 3.3890
Epoch 10/50
365/365 [==============================] - 11s 29ms/step - loss: 3.2659
Epoch 11/50
365/365 [==============================] - 11s 29ms/step - loss: 3.1434
Epoch 12/50
365/365 [==============================] - 11s 29ms/step - loss: 3.0242
E

In [12]:
model.save("model.h5")

In [13]:
import random
ind = np.random.randint(0,len(x_val)-1)

random_music = x_val[ind]

predictions=[]
for i in range(500):

    random_music = random_music.reshape(1,num_of_steps,1)

    prob  = model.predict(random_music)[0]
    y_pred= np.argmax(prob,axis=0)
    predictions.append(y_pred)

    random_music = np.insert(random_music[0],len(random_music[0]),y_pred)
    random_music = random_music[1:]
    
print(predictions)

[137, 6, 137, 6, 137, 6, 327, 103, 6, 338, 103, 249, 185, 338, 338, 338, 338, 118, 347, 338, 249, 119, 351, 351, 351, 129, 107, 23, 118, 351, 129, 129, 327, 326, 0, 107, 347, 209, 77, 77, 77, 77, 77, 177, 77, 129, 229, 177, 177, 104, 260, 103, 129, 129, 129, 103, 74, 330, 129, 177, 122, 122, 177, 177, 59, 155, 173, 155, 148, 155, 155, 104, 155, 63, 148, 148, 148, 155, 155, 347, 77, 210, 107, 155, 155, 155, 155, 155, 155, 155, 155, 155, 155, 155, 155, 108, 155, 163, 163, 163, 163, 163, 6, 163, 131, 131, 131, 6, 155, 155, 118, 118, 308, 308, 356, 155, 308, 356, 92, 356, 89, 114, 290, 173, 356, 113, 337, 89, 92, 327, 326, 172, 114, 114, 327, 173, 97, 255, 337, 308, 103, 347, 84, 347, 337, 308, 308, 308, 308, 308, 308, 308, 308, 308, 308, 308, 308, 308, 308, 308, 327, 327, 327, 260, 260, 260, 288, 260, 305, 260, 305, 260, 305, 260, 305, 260, 305, 260, 305, 260, 305, 260, 305, 260, 305, 260, 305, 260, 305, 260, 305, 260, 305, 260, 305, 260, 305, 260, 305, 260, 305, 260, 305, 260, 305, 260, 

In [14]:
x_int_to_note = dict((number, note_) for number, note_ in enumerate(unique_x)) 
predicted_notes = [x_int_to_note[i] for i in predictions]

In [15]:
def convert_to_midi(prediction_output):
   
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                
                cn=int(current_note)
                new_note = note.Note(cn)
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
                
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
            
        # pattern is a note
        else:
            
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 1
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='music.mid')

In [16]:
convert_to_midi(predicted_notes)